# Managing population growth

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt
# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")

import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [2]:
# mapping to the modules that make the app

sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")
sys.path.insert(0, "../../functions/popgrowth/")

import cmdb_graph

c = cmdb_graph.CosmosdbClient()

In [3]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"functions/popgrowth/settings.yml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'
params

{'pop_health_requirement': 0.6,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty']}

Time is updated by the `time` function. It is always running.

In [4]:
c.run_query("g.V().hasLabel('time').valueMap()")
params['time'] = c.clean_nodes(c.res)[0]
params['time']

{'currentTime': 28290,
 'updatedFrom': 'azfunction',
 'updatedAt': '2023-05-19 09:10:00.380788',
 'objid': '7263327370874',
 'name': 'time',
 'originTime': 0,
 'username': 'notebook',
 'objtype': 'time',
 'id': '7263327370874'}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

In [5]:
params['pop_health_requirement'] = 0.4

## Population Resource Consumption

In [6]:
from tools import consumption

We're going to run a resourse query test to ensure that consumption is happening. To test, update the `objid` with a planet that has consuming pops. 

In [7]:
validate_resources_updated_query = f"""
g.V().has('objid','2972908340606').out('has').valueMap()
"""
print(validate_resources_updated_query)
c.run_query(validate_resources_updated_query)
pre_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))
pre_consumption


g.V().has('objid','2972908340606').out('has').valueMap()



,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,763,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


In [8]:

all_pops_query = f"""
g.V().has('label','pop').as('pop')
    .local(
        union(
            out('inhabits').as('location'),
            out('isOf').as('species')
            )
            .fold()).as('location','species')
        .path()
        .by(unfold().valueMap().fold())
"""

In [9]:
c.run_query(all_pops_query)
data = c.reduce_res(c.res)
pd.DataFrame(data)

,pop,location,species
0,"{'name': 'Bames Todin', 'objid': '437668149746...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
1,"{'name': 'Zokekegua Long', 'objid': '664458242...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
2,"{'name': 'Bames Caslay', 'objid': '39838089911...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
3,"{'name': 'Bames Mesganj', 'objid': '8182730130...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
4,"{'name': 'Bames Fordciubeitsu', 'objid': '4068...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
5,"{'name': 'Zokekegua Bang', 'objid': '635437469...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
6,"{'name': 'Zokekegua Piakoy', 'objid': '0657651...","{'name': 'Townnak', 'class': 'terrestrial', 'o...","{'name': 'Baydybor', 'objid': '5385225260536',..."
7,"{'name': 'Obattla Magor', 'objid': '7467985401...","{'name': 'Dorf', 'class': 'terrestrial', 'obji...","{'name': 'Koykudra', 'objid': '4291284873310',..."
8,"{'name': 'Obattla Pe', 'objid': '5100638967625...","{'name': 'Dorf', 'class': 'terrestrial', 'obji...","{'name': 'Koykudra', 'objid': '4291284873310',..."
9,"{'name': 'Obattla Van', 'objid': '180095132276...","{'name': 'Dorf', 'class': 'terrestrial', 'obji...","{'name': 'Koykudra', 'objid': '4291284873310',..."


In [10]:
pops_df = pd.DataFrame([d['pop'] for d in data])
pops_df

,name,objid,conformity,literacy,aggression,constitution,health,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,Bames Todin,4376681497460,0.408,0.469,0.474,0.380,0.70,3820091516344,0.4270,0.4480,0.373,true,Billmanserver,pop,4376681497460
1,Zokekegua Long,6644582427941,0.647,0.361,0.526,0.532,0.70,4353938110450,0.5290,0.4450,0.651,true,Billmanserver,pop,6644582427941
2,Bames Caslay,3983808991145,0.394,0.496,0.448,0.602,0.70,3820091516344,0.5250,0.5105,0.361,true,Billmanserver,pop,3983808991145
3,Bames Mesganj,8182730130672,0.456,0.555,0.613,0.496,0.70,3820091516344,0.5545,0.5548,0.541,true,Billmanserver,pop,8182730130672
4,Bames Fordciubeitsu,4068023816136,0.436,0.575,0.420,0.529,0.70,3820091516344,0.4745,0.5248,0.399,true,Billmanserver,pop,4068023816136
5,Zokekegua Bang,6354374692878,0.467,0.288,0.439,0.532,0.70,4353938110450,0.4855,0.3868,0.571,true,Billmanserver,pop,6354374692878
6,Zokekegua Piakoy,0657651821206,0.570,0.461,0.428,0.548,0.70,4353938110450,0.4880,0.4745,0.575,true,Billmanserver,pop,0657651821206
7,Obattla Magor,7467985401518,0.421,0.376,0.632,0.338,0.70,8822366435138,0.4850,0.4305,0.505,true,BillmanLocal2,pop,7467985401518
8,Obattla Pe,5100638967625,0.325,0.352,0.606,0.450,0.70,8822366435138,0.5280,0.4400,0.459,true,BillmanLocal2,pop,5100638967625
9,Obattla Van,1800951322768,0.446,0.390,0.549,0.573,0.70,8822366435138,0.5610,0.4755,0.423,true,BillmanLocal2,pop,1800951322768


In [11]:
species_df = pd.DataFrame([d['species'] for d in data])
species_df

,name,objid,consumes,effuses,viral_resilience,habitat_resilience,username,objtype,id
0,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
1,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
2,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
3,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
4,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
5,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
6,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
7,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
8,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
9,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310


In [12]:
locations_df = pd.DataFrame([d['location'] for d in data])
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,isHomeworld,username,objtype,id
0,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
1,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
2,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
3,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
4,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
5,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
6,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
7,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
8,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
9,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606


First getting the list of consumption for each pop.


In [13]:
pops_df,species_df,locations_df = consumption.all_pops_consumption(c)


We expand that dataset to break open the list of people in each pop. 

In [14]:
params

{'pop_health_requirement': 0.4,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'time': {'currentTime': 28290,
  'updatedFrom': 'azfunction',
  'updatedAt': '2023-05-19 09:10:00.380788',
  'objid': '7263327370874',
  'name': 'time',
  'originTime': 0,
  'username': 'notebook',
  'objtype': 'time',
  'id': '7263327370874'}}

In [15]:
consumption_df = consumption.get_consumption_df(locations_df,species_df,params)
consumption_df = consumption.expand_consumption_df(consumption_df)
consumption_df

c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ser.consumes = j
c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ser.consumes = j
c:\Users\william.harding\Documents\repos\exoplanets\notebooks\People\../../functions/popgrowth\tools\consumption.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

,location_id,consumes,pop,consumption,multi
0,2793144513261,Organic,19,38,True
1,2816844503430,Organic,10,20,True
2,2972908340606,Organic,19,38,True
3,5640544774717,Organic,9,18,True



Then we get a list of the resources available on that location. 

In [16]:
c.run_query(consumption.make_resource_query(consumption_df))
resources = c.res
resources

[{'labels': [['location'], ['resource']],
  'objects': [{'objid': ['2793144513261'], 'name': ['Townnak']},
   {'volume': [733], 'objid': ['4741040123642'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['2972908340606'], 'name': ['Dorf']},
   {'volume': [763], 'objid': ['8679787925290'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['5640544774717'], 'name': ['Luraszi']},
   {'volume': [881], 'objid': ['4182507921316'], 'name': ['Organic']}]},
 {'labels': [['location'], ['resource']],
  'objects': [{'objid': ['2816844503430'], 'name': ['Penbufergui']},
   {'volume': [920], 'objid': ['6331044458818'], 'name': ['Organic']}]}]

In [17]:
consumption_df = consumption.tally_consumption(c,consumption_df,resources)
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,2793144513261,Organic,19,38,True,733.0,695.0
1,2816844503430,Organic,10,20,True,920.0,900.0
2,2972908340606,Organic,19,38,True,763.0,725.0
3,5640544774717,Organic,9,18,True,881.0,863.0


Then we update the resources by taking out the resources that exist. You can check that this has happened in the graph by looking at the location ids. This is a verification query that isn't run in the function application.

In [18]:
consumption_df.apply(lambda x: consumption.make_resource_update_query(c,x),axis=1)

0    None
1    None
2    None
3    None
dtype: object

In [19]:
c.run_query(validate_resources_updated_query)
post_consumption = pd.DataFrame.from_records(c.clean_nodes(c.res))

In [20]:
pre_consumption

,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,763,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


In [21]:
post_consumption

,volume,name,objid,max_volume,description,replenish_rate,username,objtype,id
0,725,Organic,8679787925290,1013,bilogical material that can be consumed by pops,10.0,BillmanLocal2,resource,8679787925290
1,96,Common Minerals,7928654046397,96,Iron and other common material used in constru...,NaN,BillmanLocal2,resource,7928654046397
2,54,Rare Minerals,3119228965862,54,"lithium, silver and other rare minerals used i...",NaN,BillmanLocal2,resource,3119228965862
3,9626,Water,1324452532458,9626,"H2O ready to be consumed, either frozen or in ...",NaN,BillmanLocal2,resource,1324452532458


Now for the starving. Populations without enough food will starve (health declines). Populations with < 0 health will die.

In [22]:
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,2793144513261,Organic,19,38,True,733.0,695.0
1,2816844503430,Organic,10,20,True,920.0,900.0
2,2972908340606,Organic,19,38,True,763.0,725.0
3,5640544774717,Organic,9,18,True,881.0,863.0


In [23]:
consumption_df[consumption_df['remaining']<=0].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

0 pops will starve in nan


,location_id,consumes,pop,consumption,multi,available,remaining


Looking more closely at `lower_health`
This only happens on locations that have < 0 resources. But here we're going to do it regardless

In [24]:
x = consumption_df.loc[0]
print(x)

dead_pop_nodes = []
dead_pop_ids = []
death_event_edges = []
query =f"""
g.V().has('objid','{x.location_id}').as('location').in('inhabits')
    .haslabel('pop').as('pop')
    .out('isOfSpecies').as('species')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('name','objid','health','username'))
        .by(valueMap('name','objid','consumes'))
"""
c.run_query(query)
out = c.res
out

location_id    2793144513261
consumes             Organic
pop                       19
consumption               38
multi                   True
available              733.0
remaining              695.0
Name: 0, dtype: object


[]

In [25]:
print(f"{len(out)} pops will starve in {x.location_id}")

0 pops will starve in 2793144513261


We are only going to starve the pop if the pop consumes that resource. 

health just goes down. 

Let's pretend that the pop will die. IF they do, it will create a `death event`.

In [26]:
# death_event = consumption.death_by_starvation_event(c.clean_node(i['objects'][0]),i['objects'][1],params)
# death_event

We will also link that event to an appropriate `happendAt` so that we can see it when looking at events related to specific things. 

In [27]:
# print(c.create_custom_edge(death_event, c.clean_node(i['objects'][0]), 'happenedAt').strip())

## Population growth
* Populations that die are deleted, so they don't grow.
* However all pops that can grow will, regardless of wheter they are starving or not. 
* Starving will lower thier health, which will eventually make them unable to grow. 


In [28]:
from tools import growth


In [29]:
pops_df,species_df,locations_df,factions_df = growth.get_pop_health(c,params)
print(f"lenth of pops == 0 : {len(pops_df)==0}")
pops_df

lenth of pops == 0 : False


,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,0.7,Bames Todin,4376681497460,0.408,0.469,0.474,0.380,3820091516344,0.4270,0.4480,0.373,true,Billmanserver,pop,4376681497460
1,0.7,Zokekegua Long,6644582427941,0.647,0.361,0.526,0.532,4353938110450,0.5290,0.4450,0.651,true,Billmanserver,pop,6644582427941
2,0.7,Bames Caslay,3983808991145,0.394,0.496,0.448,0.602,3820091516344,0.5250,0.5105,0.361,true,Billmanserver,pop,3983808991145
3,0.7,Bames Mesganj,8182730130672,0.456,0.555,0.613,0.496,3820091516344,0.5545,0.5548,0.541,true,Billmanserver,pop,8182730130672
4,0.7,Bames Fordciubeitsu,4068023816136,0.436,0.575,0.420,0.529,3820091516344,0.4745,0.5248,0.399,true,Billmanserver,pop,4068023816136
5,0.7,Zokekegua Bang,6354374692878,0.467,0.288,0.439,0.532,4353938110450,0.4855,0.3868,0.571,true,Billmanserver,pop,6354374692878
6,0.7,Zokekegua Piakoy,0657651821206,0.570,0.461,0.428,0.548,4353938110450,0.4880,0.4745,0.575,true,Billmanserver,pop,0657651821206
7,0.7,Obattla Magor,7467985401518,0.421,0.376,0.632,0.338,8822366435138,0.4850,0.4305,0.505,true,BillmanLocal2,pop,7467985401518
8,0.7,Obattla Pe,5100638967625,0.325,0.352,0.606,0.450,8822366435138,0.5280,0.4400,0.459,true,BillmanLocal2,pop,5100638967625
9,0.7,Obattla Van,1800951322768,0.446,0.390,0.549,0.573,8822366435138,0.5610,0.4755,0.423,true,BillmanLocal2,pop,1800951322768


In [30]:
species_df

,name,objid,consumes,effuses,viral_resilience,habitat_resilience,username,objtype,id
0,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
1,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
2,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
3,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
4,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
5,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
6,Baydybor,5385225260536,[Organic],"[Organic waste, Plastics]",0.7,0.2,Billmanserver,species,5385225260536
7,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
8,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310
9,Koykudra,4291284873310,[Organic],"[Organic waste, Plastics]",0.7,0.2,BillmanLocal2,species,4291284873310


In [31]:
locations_df

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,isHomeworld,username,objtype,id
0,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
1,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
2,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
3,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
4,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
5,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
6,Townnak,terrestrial,2793144513261,0.701,0.884,0.490,6092232618401,Woodling,true,true,true,Billmanserver,planet,2793144513261
7,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
8,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606
9,Dorf,terrestrial,2972908340606,0.884,0.882,1.057,9570931993836,Lisallos,true,true,true,BillmanLocal2,planet,2972908340606


In [32]:
factions_df

,pop_locations,name,objid,lat,long,pop_loactions,username,objtype,id
0,"[[0, 0], [-2, -2], [0, 2], [-2, 2]]",Bames,3820091516344,0.105,0.0,"[[0, 0], [-1, -1], [-2, 0], [0, 2], [-1, 1]]",Billmanserver,faction,3820091516344
1,"[[0, 0], [-2, 2], [-1, 1], [1, 1]]",Zokekegua,4353938110450,-0.105,0.0,"[[0, 0], [2, 0], [-2, 0], [0, -2]]",Billmanserver,faction,4353938110450
2,"[[0, 0], [-2, -2], [0, 2], [-2, 2]]",Bames,3820091516344,0.105,0.0,"[[0, 0], [-1, -1], [-2, 0], [0, 2], [-1, 1]]",Billmanserver,faction,3820091516344
3,"[[0, 0], [-2, -2], [0, 2], [-2, 2]]",Bames,3820091516344,0.105,0.0,"[[0, 0], [-1, -1], [-2, 0], [0, 2], [-1, 1]]",Billmanserver,faction,3820091516344
4,"[[0, 0], [-2, -2], [0, 2], [-2, 2]]",Bames,3820091516344,0.105,0.0,"[[0, 0], [-1, -1], [-2, 0], [0, 2], [-1, 1]]",Billmanserver,faction,3820091516344
5,"[[0, 0], [-2, 2], [-1, 1], [1, 1]]",Zokekegua,4353938110450,-0.105,0.0,"[[0, 0], [2, 0], [-2, 0], [0, -2]]",Billmanserver,faction,4353938110450
6,"[[0, 0], [-2, 2], [-1, 1], [1, 1]]",Zokekegua,4353938110450,-0.105,0.0,"[[0, 0], [2, 0], [-2, 0], [0, -2]]",Billmanserver,faction,4353938110450
7,"[[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...",Obattla,8822366435138,-0.149,0.0,NaN,BillmanLocal2,faction,8822366435138
8,"[[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...",Obattla,8822366435138,-0.149,0.0,NaN,BillmanLocal2,faction,8822366435138
9,"[[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...",Obattla,8822366435138,-0.149,0.0,NaN,BillmanLocal2,faction,8822366435138


In [33]:
factions_df['pop_locations']

0                   [[0, 0], [-2, -2], [0, 2], [-2, 2]]
1                    [[0, 0], [-2, 2], [-1, 1], [1, 1]]
2                   [[0, 0], [-2, -2], [0, 2], [-2, 2]]
3                   [[0, 0], [-2, -2], [0, 2], [-2, 2]]
4                   [[0, 0], [-2, -2], [0, 2], [-2, 2]]
5                    [[0, 0], [-2, 2], [-1, 1], [1, 1]]
6                    [[0, 0], [-2, 2], [-1, 1], [1, 1]]
7     [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...
8     [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...
9     [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...
10    [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...
11                  [[0, 0], [-2, 2], [-1, -1], [2, 0]]
12    [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...
13    [[0, 0], [-1, 1], [-2, 0], [0, 2], [-1, -1], [...
14                           [[0, 0], [-2, 0], [1, -1]]
15                           [[0, 0], [-2, 0], [1, -1]]
16                           [[0, 0], [0, -2], [2, -2]]
17                           [[0, 0], [-2, 0], [

In [34]:
pops_df['roll'] = pops_df['objid'].apply(lambda x: np.random.random())
pops_df['grow'] = pops_df[['wealth','health']].T.mean() >= pops_df['roll']

reproducing_pops = pops_df[pops_df['grow']].drop(['roll','grow'],axis=1)
pops_df

,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id,roll,grow
0,0.7,Bames Todin,4376681497460,0.408,0.469,0.474,0.380,3820091516344,0.4270,0.4480,0.373,true,Billmanserver,pop,4376681497460,0.169464,True
1,0.7,Zokekegua Long,6644582427941,0.647,0.361,0.526,0.532,4353938110450,0.5290,0.4450,0.651,true,Billmanserver,pop,6644582427941,0.133566,True
2,0.7,Bames Caslay,3983808991145,0.394,0.496,0.448,0.602,3820091516344,0.5250,0.5105,0.361,true,Billmanserver,pop,3983808991145,0.153874,True
3,0.7,Bames Mesganj,8182730130672,0.456,0.555,0.613,0.496,3820091516344,0.5545,0.5548,0.541,true,Billmanserver,pop,8182730130672,0.358523,True
4,0.7,Bames Fordciubeitsu,4068023816136,0.436,0.575,0.420,0.529,3820091516344,0.4745,0.5248,0.399,true,Billmanserver,pop,4068023816136,0.526387,True
5,0.7,Zokekegua Bang,6354374692878,0.467,0.288,0.439,0.532,4353938110450,0.4855,0.3868,0.571,true,Billmanserver,pop,6354374692878,0.800228,False
6,0.7,Zokekegua Piakoy,0657651821206,0.570,0.461,0.428,0.548,4353938110450,0.4880,0.4745,0.575,true,Billmanserver,pop,0657651821206,0.918243,False
7,0.7,Obattla Magor,7467985401518,0.421,0.376,0.632,0.338,8822366435138,0.4850,0.4305,0.505,true,BillmanLocal2,pop,7467985401518,0.421370,True
8,0.7,Obattla Pe,5100638967625,0.325,0.352,0.606,0.450,8822366435138,0.5280,0.4400,0.459,true,BillmanLocal2,pop,5100638967625,0.074967,True
9,0.7,Obattla Van,1800951322768,0.446,0.390,0.549,0.573,8822366435138,0.5610,0.4755,0.423,true,BillmanLocal2,pop,1800951322768,0.888872,False


In [35]:
reproducing_pops

,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,0.7,Bames Todin,4376681497460,0.408,0.469,0.474,0.380,3820091516344,0.4270,0.4480,0.373,true,Billmanserver,pop,4376681497460
1,0.7,Zokekegua Long,6644582427941,0.647,0.361,0.526,0.532,4353938110450,0.5290,0.4450,0.651,true,Billmanserver,pop,6644582427941
2,0.7,Bames Caslay,3983808991145,0.394,0.496,0.448,0.602,3820091516344,0.5250,0.5105,0.361,true,Billmanserver,pop,3983808991145
3,0.7,Bames Mesganj,8182730130672,0.456,0.555,0.613,0.496,3820091516344,0.5545,0.5548,0.541,true,Billmanserver,pop,8182730130672
4,0.7,Bames Fordciubeitsu,4068023816136,0.436,0.575,0.420,0.529,3820091516344,0.4745,0.5248,0.399,true,Billmanserver,pop,4068023816136
7,0.7,Obattla Magor,7467985401518,0.421,0.376,0.632,0.338,8822366435138,0.4850,0.4305,0.505,true,BillmanLocal2,pop,7467985401518
8,0.7,Obattla Pe,5100638967625,0.325,0.352,0.606,0.450,8822366435138,0.5280,0.4400,0.459,true,BillmanLocal2,pop,5100638967625
10,0.7,Obattla Rochi,0692394121482,0.452,0.395,0.514,0.546,8822366435138,0.5300,0.4625,0.354,true,BillmanLocal2,pop,0692394121482
13,0.7,Obattla Ser,7828874183454,0.372,0.370,0.575,0.581,8822366435138,0.5780,0.4740,0.424,true,BillmanLocal2,pop,7828874183454
15,0.7,Ganjtil Wo,9552341693154,0.494,0.615,0.354,0.410,4115897493342,0.3820,0.4985,0.485,true,manderson,pop,9552341693154


In [36]:
nodes = []
edges = []
# Event edges must be uploaded separately, because the event nodes haven't been created yet.    
event_edges = []

In [37]:

for i in reproducing_pops.index.to_list():
    p = reproducing_pops.loc[i].to_dict()
    f = factions_df.loc[i].to_dict()
    species = species_df.loc[i].to_dict()
    location = locations_df.loc[i].to_dict()
    child = growth.grow_pop(p,species,params,syllables)    
    pop_locations = growth.assign_pop_to_faction(f)    
    c.patch_property(f['objid'], 'pop_locations', str(pop_locations))
    nodes.append(child)
    event = growth.population_growth_event(p, location,child, params)
    nodes.append(event)
    edges.append({"node1": child["objid"], "node2": p["objid"], "label": "childOf"})
    edges.append({"node1": child["objid"], "node2": child["isIn"], "label": "isIn"})
    edges.append({"node1": child["objid"], "node2": species["objid"], "label": "isOf"})
    edges.append({"node1": child["objid"], "node2": location["objid"], "label": "inhabits"})
    event_edges.append(c.create_custom_edge(event,location,'happenedAt'))
    event_edges.append(c.create_custom_edge(p,event,'caused'))


In [38]:
f

{'pop_locations': '[[0, 0], [2, -2]]',
 'name': 'Tilcasport',
 'objid': '4232678858631',
 'lat': 0.0,
 'long': 0.0,
 'pop_loactions': '[[0, 0], [0, 2], [-2, 0], [-2, 2], [2, 0], [2, -2], [3, 1], [1, 1]]',
 'username': 'william.jeffrey.harding@gmail.com',
 'objtype': 'faction',
 'id': '4232678858631'}

In [39]:
pop_locations = growth.assign_pop_to_faction(f) 
pop_locations


[[0, 0], [2, -2], [-2, -2]]

In [40]:
c.patch_property(f['objid'], 'pop_locations', str(pop_locations))

In [41]:
c.run_query(f"""
g.V().has('objid','{f['objid']}')
""")    

c.res

[{'id': '4232678858631',
  'label': 'faction',
  'type': 'vertex',
  'properties': {'pop_locations': [{'id': '12b1427f-90b9-4cf4-a4fc-5e60a8137542',
     'value': '[[0, 0], [2, -2], [-2, -2]]'}],
   'name': [{'id': 'f56d227e-5c85-4d4b-b219-d69e37fb7553',
     'value': 'Tilcasport'}],
   'objid': [{'id': '4232678858631|objid', 'value': '4232678858631'}],
   'lat': [{'id': '28c5a6f4-7fc0-43a1-b0f0-7b8320064d7e', 'value': 0}],
   'long': [{'id': '86e4814f-e44b-487a-870d-c93dcd08cfa5', 'value': 0}],
   'pop_loactions': [{'id': '3452b943-27c1-4e60-896f-38827c0e07e8',
     'value': '[[0, 0], [0, 2], [-2, 0], [-2, 2], [2, 0], [2, -2], [3, 1], [1, 1]]'}],
   'username': [{'id': '73ec223a-2576-466c-a3bc-03a83ef18774',
     'value': 'william.jeffrey.harding@gmail.com'}],
   'objtype': [{'id': '1ef5c86a-0b5d-438a-9214-fc362d781b42',
     'value': 'faction'}]}}]

In [38]:
p

{'health': 0.7,
 'name': 'Tilcasport Enwil',
 'objid': '3440955961419',
 'conformity': 0.683,
 'literacy': 0.615,
 'aggression': 0.373,
 'constitution': 0.228,
 'isIn': '4232678858631',
 'industry': 0.3005,
 'wealth': 0.4578,
 'factionLoyalty': 0.802,
 'isIdle': 'true',
 'username': 'william.jeffrey.harding@gmail.com',
 'objtype': 'pop',
 'id': '3440955961419'}

In [40]:
growth.grow_pop(p,species,params,syllables)

{'health': 0.42,
 'name': 'Tilcasport Enwilral',
 'objid': '5618847005479',
 'conformity': 0.719,
 'literacy': 0.557,
 'aggression': 0.341,
 'constitution': 0.208,
 'isIn': '4232678858631',
 'industry': 0.18,
 'wealth': 0.221,
 'factionLoyalty': 0.798,
 'isIdle': 'true',
 'username': 'william.jeffrey.harding@gmail.com',
 'objtype': 'pop',
 'id': '5618847005479',
 'label': 'pop'}

In [41]:
pd.DataFrame.from_records(nodes)


,health,name,objid,conformity,literacy,aggression,constitution,isIn,industry,wealth,factionLoyalty,isIdle,username,objtype,id,label,text,visibleTo,time
0,0.42,Zokekegua Longken,3875124605399,0.605,0.401,0.431,0.444,4353938110450,0.317,0.337,0.694,true,Billmanserver,pop,3875124605399,pop,NaN,NaN,NaN
1,NaN,population growth,3741412120711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,event,The population (Zokekegua Long) inhabiting Tow...,Billmanserver,28290.0
2,0.42,Bames Mesganjma,4460605932082,0.521,0.551,0.626,0.588,3820091516344,0.333,0.279,0.484,true,Billmanserver,pop,4460605932082,pop,NaN,NaN,NaN
3,NaN,population growth,9394323283459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,event,The population (Bames Mesganj) inhabiting Town...,Billmanserver,28290.0
4,0.42,Zokekegua Banglet,2370990554869,0.508,0.260,0.406,0.574,4353938110450,0.291,0.160,0.630,true,Billmanserver,pop,2370990554869,pop,NaN,NaN,NaN
5,NaN,population growth,9914342017362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,event,The population (Zokekegua Bang) inhabiting Tow...,Billmanserver,28290.0
6,0.42,Obattla Pehill,7824269356496,0.362,0.416,0.567,0.492,8822366435138,0.317,0.190,0.548,true,BillmanLocal2,pop,7824269356496,pop,NaN,NaN,NaN
7,NaN,population growth,7786533121394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,event,The population (Obattla Pe) inhabiting Dorf ha...,BillmanLocal2,28290.0
8,0.42,Obattla Vanter,5973691014700,0.400,0.402,0.495,0.605,8822366435138,0.337,0.217,0.425,true,BillmanLocal2,pop,5973691014700,pop,NaN,NaN,NaN
9,NaN,population growth,2978622812759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,event,NaN,NaN,event,The population (Obattla Van) inhabiting Dorf h...,BillmanLocal2,28290.0


In [42]:
pd.DataFrame.from_records(edges)

,node1,node2,label
0,3875124605399,6644582427941,childOf
1,3875124605399,4353938110450,isIn
2,3875124605399,5385225260536,isOf
3,3875124605399,2793144513261,inhabits
4,4460605932082,8182730130672,childOf
...,...,...,...
71,9132476604075,2816844503430,inhabits
72,0944104855818,3440955961419,childOf
73,0944104855818,4232678858631,isIn
74,0944104855818,1245644824657,isOf


In [43]:
event_edges

["\n        g.V().has('objid','3741412120711')\n            .addE('happenedAt')\n            .to(g.V().has('objid','2793144513261'))\n        ",
 "\n        g.V().has('objid','6644582427941')\n            .addE('caused')\n            .to(g.V().has('objid','3741412120711'))\n        ",
 "\n        g.V().has('objid','9394323283459')\n            .addE('happenedAt')\n            .to(g.V().has('objid','2793144513261'))\n        ",
 "\n        g.V().has('objid','8182730130672')\n            .addE('caused')\n            .to(g.V().has('objid','9394323283459'))\n        ",
 "\n        g.V().has('objid','9914342017362')\n            .addE('happenedAt')\n            .to(g.V().has('objid','2793144513261'))\n        ",
 "\n        g.V().has('objid','6354374692878')\n            .addE('caused')\n            .to(g.V().has('objid','9914342017362'))\n        ",
 "\n        g.V().has('objid','7786533121394')\n            .addE('happenedAt')\n            .to(g.V().has('objid','2972908340606'))\n        

In [44]:
growth.grow(c,params,syllables)